In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyautogui as p
import time
import requests

### Grab player data using fetchplayerstats class

In [ ]:
from NBAData.fetchPlayersStats import FetchPlayersStats

nba = FetchPlayersStats()
data = nba.getCompleteStats(season='2024-25', season_type='Playoffs', sleep_time=1, max_workers=5)
data

In [3]:
data.to_csv('PLAYOFF_DATA/PLAYOFFS_25.csv')

In [2]:
pd.set_option('display.max_columns', None)
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25.csv')
data = data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
data['OPP_TEAM_ID'] = data['OPP_TEAM_ID'].fillna(0).astype(int)
data.head()

,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,OFF_RATING,DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,PACE_PER40,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE
0,AJ Green,1631260,MIL @ IND,MIL,1610612749,IND,0,42400131,2025-04-19,L,27,15,1,3,5,13,0.385,5,11,0.455,0,0,NaN,0,3,0,0,0,5,-5,20.1,1.154,1.154,NaN,100.0,107.3,-7.3,0.000,0.111,0.054,0.071,0.0,0.220,0.577,94.06,97.35,0.065,81.12,"APR 19, 2025",240,35,84,0.417,9,37,0.243,19,28,0.679,8,33,41,15,4,3,10,20,98,99.674532,6.0,6.0,43.0,0.518519,1610612754,99.674532,98.32,95.38,92.44
1,Rudy Gobert,203497,MIN @ LAL,MIN,1610612750,LAL,0,42400161,2025-04-19,W,24,2,1,6,1,2,0.500,0,0,NaN,0,0,NaN,1,5,0,1,0,3,14,13.7,1.000,1.000,C,128.3,95.7,32.5,0.037,0.185,0.111,0.045,0.0,0.037,0.500,93.29,91.79,0.047,76.49,"APR 19, 2025",240,44,86,0.512,21,42,0.500,8,10,0.800,11,33,44,29,8,4,10,21,117,130.872483,5.0,4.0,38.0,0.397590,1610612747,130.872483,89.40,89.72,90.04
2,Shake Milton,1629003,LAL vs. MIN,LAL,1610612747,MIN,1,42400161,2025-04-19,L,3,0,0,0,0,1,0.000,0,0,NaN,0,0,NaN,0,0,0,0,0,1,1,0.0,0.000,0.000,NaN,125.0,80.0,45.0,0.000,0.000,0.000,0.000,0.0,0.111,0.000,62.35,71.21,-0.400,59.34,"APR 19, 2025",240,33,83,0.398,15,41,0.366,14,16,0.875,13,25,38,15,5,4,13,10,95,105.508663,8.0,4.0,44.0,0.511628,1610612750,105.508663,90.04,89.72,89.40
3,Jarred Vanderbilt,1629020,LAL vs. MIN,LAL,1610612747,MIN,1,42400161,2025-04-19,L,14,2,2,4,1,3,0.333,0,0,NaN,0,0,NaN,2,2,1,0,0,0,-2,12.8,0.667,0.667,NaN,141.7,150.0,-8.3,0.182,0.200,0.190,0.182,0.0,0.107,0.333,86.17,85.25,0.088,71.04,"APR 19, 2025",240,33,83,0.398,15,41,0.366,14,16,0.875,13,25,38,15,5,4,13,10,95,105.508663,8.0,4.0,44.0,0.511628,1610612750,105.508663,90.04,89.72,89.40
4,Rui Hachimura,1629060,LAL vs. MIN,LAL,1610612747,MIN,1,42400161,2025-04-19,L,30,9,2,5,3,9,0.333,3,7,0.429,0,0,NaN,0,5,0,0,1,0,-26,17.0,1.000,1.000,F,79.3,126.3,-47.0,0.000,0.179,0.081,0.167,2.0,0.154,0.500,93.12,93.09,0.087,77.57,"APR 19, 2025",240,33,83,0.398,15,41,0.366,14,16,0.875,13,25,38,15,5,4,13,10,95,105.508663,8.0,4.0,44.0,0.511628,1610612750,105.508663,90.04,89.72,89.40


In [ ]:
from NBAData.features import *
from NBAData.playerPositions import *

data['STARTING'] = data['START_POSITION'].apply(lambda x: 1 if x in ['G','F','C'] else 0)
data = add_PTS_features(data)
data = addMinRollingAvgs(data)
data = addFgaRollingAvgs(data)
data = calculate_days_of_rest(data)
data = assign_playoff_series_info(data)
data = assign_position(data, max_workers=4, delay_between_requests=1)
data = assign_position_with_cache(
    data, 
    cache_file='nba_positions.csv',
    max_workers=4, 
    delay_between_requests=1.5
)
data

Extracting unique player IDs...
Found 219 unique players to process...
Fetching player positions using 4 threads...
Gathered data for PLAYER_ID 201144... (1/219)
Gathered data for PLAYER_ID 201143... (2/219)
Gathered data for PLAYER_ID 200782... (3/219)
Gathered data for PLAYER_ID 2544... (4/219)
Gathered data for PLAYER_ID 201145... (5/219)
Gathered data for PLAYER_ID 201566... (6/219)
Gathered data for PLAYER_ID 201572... (7/219)
Gathered data for PLAYER_ID 201587... (8/219)
Gathered data for PLAYER_ID 201599... (9/219)
Gathered data for PLAYER_ID 201935... (10/219)
Progress: 10/219 players processed
Gathered data for PLAYER_ID 201949... (11/219)
Gathered data for PLAYER_ID 201939... (12/219)
Gathered data for PLAYER_ID 201950... (13/219)
Gathered data for PLAYER_ID 201988... (14/219)
Gathered data for PLAYER_ID 202684... (15/219)
Gathered data for PLAYER_ID 202692... (16/219)
Gathered data for PLAYER_ID 202699... (17/219)
Gathered data for PLAYER_ID 202695... (18/219)
Gathered data 

,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,OFF_RATING,DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,PACE_PER40,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE,STARTING,USG_PCT_LAST_3,USG_PCT_LAST_5,USG_PCT_LAST_7,PTS_LAST_3,PTS_LAST_5,PTS_LAST_7,STD_PTS_LAST_3,STD_PTS_LAST_5,STD_PTS_LAST_7,PLAYER_HOME_AVG_PTS,PLAYER_AWAY_AVG_PTS,MIN_LAST_2,MIN_LAST_4,MIN_LAST_6,FGA_LAST_2,FGA_LAST_4,FGA_LAST_6,DAYS_OF_REST,GUARD,FORWARD,CENTER
43,LeBron James,2544,LAL vs. MIN,LAL,1610612747,MIN,1,42400161,2025-04-19,L,36,19,3,5,8,18,0.444,1,5,0.200,2,2,1.000,1,4,2,3,4,0,-22,40.5,1.006,1.006,F,108.7,142.6,-34.0,0.029,0.138,0.078,0.158,0.75,0.278,0.503,90.65,90.70,0.099,75.59,"APR 19, 2025",240,33,83,0.398,15,41,0.366,14,16,0.875,13,25,38,15,5,4,13,10,95,105.508663,8.0,4.0,44.0,0.511628,1610612750,105.508663,90.04,89.72,89.40,1,0.28,0.28,0.28,19.00,19.00,19.00,NaN,NaN,NaN,19.00,NaN,36.0,36.00,36.00,18.0,18.00,18.00,NaN,0,1,0
245,LeBron James,2544,LAL vs. MIN,LAL,1610612747,MIN,1,42400162,2025-04-22,W,40,21,7,11,8,19,0.421,1,5,0.200,4,4,1.000,3,8,1,1,2,4,9,48.7,1.012,1.012,F,111.4,98.6,12.9,0.094,0.195,0.151,0.368,3.50,0.284,0.506,88.21,84.81,0.224,70.68,"APR 22, 2025",240,34,75,0.453,6,29,0.207,20,20,1.000,9,32,41,23,4,6,12,24,94,108.294931,6.0,2.0,34.0,0.379747,1610612750,108.294931,86.80,89.40,92.00,1,0.28,0.28,0.28,20.00,20.00,20.00,1.41,1.41,1.41,20.00,NaN,38.0,38.00,38.00,18.5,18.50,18.50,3.0,0,1,0
450,LeBron James,2544,LAL @ MIN,LAL,1610612747,MIN,0,42400163,2025-04-25,L,41,38,4,10,13,21,0.619,5,9,0.556,7,10,0.700,3,7,2,2,1,0,-10,67.0,1.496,1.496,F,109.9,117.9,-8.0,0.077,0.179,0.128,0.211,4.00,0.286,0.748,98.35,96.23,0.266,80.19,"APR 25, 2025",240,37,78,0.474,17,40,0.425,13,20,0.650,7,31,38,21,5,7,16,23,104,108.559499,10.0,5.0,38.0,0.483516,1610612750,108.559499,95.80,97.52,99.24,1,0.28,0.28,0.28,26.00,26.00,26.00,10.44,10.44,10.44,NaN,38.00,40.5,39.00,39.00,20.0,19.33,19.33,3.0,0,1,0
609,LeBron James,2544,LAL @ MIN,LAL,1610612747,MIN,0,42400164,2025-04-27,L,46,27,8,12,5,9,0.556,2,4,0.500,15,18,0.833,3,9,3,3,3,2,-4,68.4,1.596,1.596,F,119.1,125.0,-5.9,0.070,0.158,0.120,0.276,2.67,0.186,0.798,92.80,91.88,0.207,76.57,"APR 27, 2025",240,36,80,0.450,19,47,0.404,22,25,0.880,11,30,41,23,6,7,10,23,113,125.555556,8.0,2.0,49.0,0.422222,1610612750,125.555556,90.00,92.76,95.52,1,0.25,0.26,0.26,28.67,26.25,26.25,8.62,8.54,8.54,NaN,32.50,43.5,40.75,40.75,15.0,16.75,16.75,2.0,0,1,0
788,LeBron James,2544,LAL vs. MIN,LAL,1610612747,MIN,1,42400165,2025-04-30,L,40,22,6,7,9,21,0.429,1,5,0.200,3,6,0.500,2,5,2,0,3,2,-14,42.4,0.931,0.931,F,93.8,112.7,-18.9,0.049,0.111,0.081,0.375,2.00,0.292,0.465,95.53,94.34,0.132,78.62,"APR 30, 2025",240,33,78,0.423,11,37,0.297,19,25,0.760,8,29,37,24,5,1,15,25,96,100.000000,8.0,5.0,54.0,0.404494,1610612750,100.000000,96.00,96.26,96.52,1,0.25,0.27,0.27,29.00,25.40,25.40,8.19,7.64,7.64,20.67,NaN,43.0,41.75,40.60,15.0,17.50,17.60,3.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,Quinten Post,1642366,GSW @ MIN,GSW,1610612744,MIN,0,42400231,2025-05-06,W,6,0,0,1,0,0,NaN,0,0,NaN,0,0,NaN,0,1,0,0,0,0,14,1.2,0.000,0.000,NaN,138.5,28.6,109.9,0.000,0.111,0.071,0.000,0.00,0.0

In [6]:
data.to_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')